<a href="https://colab.research.google.com/github/SDCoulter/big_data_nlp/blob/main/03_PySpark_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Latest version of Spark.
spark_version = 'spark-3.1.2/'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java.
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

# Set environment variables.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.1.2-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
# Start SparkSession.
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [3]:
# Import Tokenizer library.
from pyspark.ml.feature import Tokenizer

In [14]:
# Create sample DataFrame.
df = spark.createDataFrame([
                            (0, "Spark is great"),
                            (1, "We are learning Spark"),
                            (2, "Spark is better than Hadoop")
], ['id', 'sentence'])

df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [6]:
# Tokenize sentences.
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")


Tokenizer_fb0b393dd04f

In [7]:
# Transform and show DF.
tokenized_df = tokenizer.transform(df)
tokenized_df.show(truncate=False)

+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|0  |Spark is great                      |[spark, is, great]                          |
|1  |We are learning Spark               |[we, are, learning, spark]                  |
|2  |Spark is better than Hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|
+---+------------------------------------+--------------------------------------------+



In [8]:
# Create the word count function.
def word_list_length(word_list):
  return len(word_list)

In [9]:
# Import the udf function to add our word count function, and the col function
# to select a column to be passed in.
from pyspark.sql.functions import col, udf
# Import the data type that matches the output of the function.
from pyspark.sql.types import IntegerType

In [10]:
# Create a UDF.
count_tokens = udf(word_list_length, IntegerType())

In [15]:
# Create the new tokenizer.
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

# Transform DataFrame.
tokenized_df = tokenizer.transform(df)

# Select the needed columns and don't truncate the results.
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+---------------------------+---------------------------------+------+
|id |sentence                   |words                            |tokens|
+---+---------------------------+---------------------------------+------+
|0  |Spark is great             |[spark, is, great]               |3     |
|1  |We are learning Spark      |[we, are, learning, spark]       |4     |
|2  |Spark is better than Hadoop|[spark, is, better, than, hadoop]|5     |
+---+---------------------------+---------------------------------+------+

